In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
import jieba
import codecs

In [8]:
def cut(string, stop_words=None):
    """
    分词
    :param string: 待分词的句子
    :return: 分词所得的列表
    """
    words = list(jieba.cut(string.strip()))
    words_return = []
    if stop_words:
        for word in words:
            if word not in stop_words:
                words_return.append(word)
    return words_return

In [9]:
def get_stop_word_list(filename="../data/hlt_stop_words.txt"):
    """
    返回停词表
    :param filename: 停词表位置
    :return: <List> 停词表
    """
    stop_words = []
    with codecs.open(filename, "r", "utf=8") as stop_word_file:
        for line in stop_word_file:
            stop_words.append(line.strip())
    return stop_words

In [10]:
def get_data(filename="../data/train/train.csv"):
    """
    获取训练或测试数据集
    :param filename: 数据位置
    :return: <pandas.DataFrame> 返回一个pandas表
    """
    data = pd.read_csv(filename)
    return data

In [11]:
train_data = get_data("../data/train/train.csv")
test_data = get_data("../data/test_public/test_public.csv")

In [24]:
stop_words = get_stop_word_list()
cut_words_train = []
for ind in train_data.index:
    sentence = train_data.loc[ind, "content"]
    words = cut(sentence, stop_words=stop_words)
    cut_words_train.append(words)
    train_data.loc[ind, "content"] = " ".join(words)
    print("\rProcess: {:5d}/{:5d}".format(ind, train_data.shape[0]-1), end="")

print('\n')

cut_words_test = []
for ind in test_data.index:
    sentence = test_data.loc[ind, "content"]
    words = cut(sentence, stop_words=stop_words)
    cut_words_test.append(words)
    test_data.loc[ind, "content"] = " ".join(words)
    print("\rProcess: {:5d}/{:5d}".format(ind, test_data.shape[0]-1), end="")

Process:  9946/ 9946

Process:  2363/ 2363

In [20]:
categories = ['价格', '内饰', '动力', '外观', '安全性', '操控', '油耗', '空间', '舒适性', '配置']

x_train, x_test, y_train, y_test = train_test_split(train_data['content'].values, train_data['subject'].values, test_size=0.2, random_state=2018)

# lb = LabelEncoder()
y = train_data['subject'].values.ravel()
y_train = y_train.ravel()
y_test = y_test.ravel()

print("Train/Test split: {:d}/{:d}".format(len(y_train), len(y_test)))

Train/Test split: 7957/1990


In [21]:
""" Support Vector Machine (SVM) classifier"""
svm_clf = Pipeline([('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter= 5, random_state=42)),
])
svm_clf.fit(x_train, y_train)
""" Predict the test dataset using Naive Bayes"""
predicted = svm_clf.predict(x_test)
print('SVM correct prediction: {:4.4f}'.format(np.mean(predicted == y_test)))
print(metrics.classification_report(y_test, predicted, target_names=categories))

print("Confusion Matrix:")
print(metrics.confusion_matrix(y_test, predicted))
print('\n')

SVM correct prediction: 0.6975
             precision    recall  f1-score   support

         价格       0.75      0.88      0.81       241
         内饰       0.64      0.42      0.51       102
         动力       0.65      0.86      0.74       532
         外观       0.70      0.52      0.59        91
        安全性       0.65      0.60      0.62       126
         操控       0.63      0.49      0.55       228
         油耗       0.79      0.81      0.80       225
         空间       0.57      0.53      0.55        87
        舒适性       0.74      0.52      0.61       184
         配置       0.82      0.70      0.75       174

avg / total       0.70      0.70      0.69      1990

Confusion Matrix:
[[211   0  20   1   2   4   0   0   0   3]
 [  8  43  15   5   6   8   2   4   6   5]
 [ 17   1 457   2   3  14  18   6  10   4]
 [  5   3  24  47   1   6   1   1   1   2]
 [  3   0  26   0  75  14   3   1   2   2]
 [ 11   6  50   3  20 111   7   8   8   4]
 [  6   1  30   0   0   3 182   2   1   0]
 [  5   2  

In [22]:
""" 10-fold cross vaildation """
clf_s= make_pipeline(CountVectorizer(), TfidfTransformer(), SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter= 5, random_state=42))
svm_10_fold = cross_val_score(clf_s, train_data['content'].values, train_data['subject'].values, cv=10)
print('SVM 10-fold correct prediction: {:4.4f}'.format(np.mean(svm_10_fold)))

SVM 10-fold correct prediction: 0.7072


In [25]:
test_subject_pred = svm_clf.predict(test_data['content'].values)

In [26]:
test_subject_pred

array(['价格', '价格', '价格', ..., '配置', '动力', '安全性'],
      dtype='<U3')

In [41]:
categories = ['-1', '0', '1']
x_train, x_test, y_train, y_test = train_test_split(train_data["content"].values, 
                                                    train_data["sentiment_value"].values, 
                                                    test_size=0.2)

In [47]:
""" Support Vector Machine (SVM) classifier"""
svm_clf = Pipeline([('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter= 5, random_state=42)),
#     ('clf', SVC()),
])
svm_clf.fit(x_train, y_train)
""" Predict the test dataset using Naive Bayes"""
predicted = svm_clf.predict(x_test)
print('SVM correct prediction: {:4.4f}'.format(np.mean(predicted == y_test)))
print(metrics.classification_report(y_test, predicted, target_names=categories))

print("Confusion Matrix:")
print(metrics.confusion_matrix(y_test, predicted))
print('\n')

SVM correct prediction: 0.6688
             precision    recall  f1-score   support

         -1       0.00      0.00      0.00       320
          0       0.67      1.00      0.80      1330
          1       1.00      0.00      0.01       340

avg / total       0.62      0.67      0.54      1990

Confusion Matrix:
[[   0  320    0]
 [   0 1330    0]
 [   0  339    1]]




D:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [48]:
""" 10-fold cross vaildation """
clf_s= make_pipeline(CountVectorizer(), TfidfTransformer(), SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter= 5, random_state=42))
svm_10_fold = cross_val_score(clf_s, train_data['content'].values, train_data['sentiment_value'].values, cv=10)
print('SVM 10-fold correct prediction: {:4.4f}'.format(np.mean(svm_10_fold)))

SVM 10-fold correct prediction: 0.6696


In [36]:
test_sentiment_value_pred = svm_clf.predict(test_data['content'].values)

In [37]:
test_sentiment_value_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [39]:
with codecs.open("../output/output.csv", "w", "utf-8") as outfile:
    outfile.write("content_id,subject,sentiment_value,sentiment_word\n")
    cnt = 1
    for content_id, subject, sentiment_value in zip(test_data["content_id"], test_subject_pred, test_sentiment_value_pred):
        outfile.write("{},{},{},\n".format(content_id, subject, sentiment_value))
        print("\rProcess: {}/{}".format(cnt, test_data.index[-1]+1), end="")
        cnt += 1

Process: 822/2364

Process: 2364/2364